In [1]:
import uvicorn 
import gensim
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from fastapi import FastAPI
from fastapi import Header
from pydantic import BaseModel
import pandas as pd 
import numpy as np 
from gensim.test.utils import datapath
from ast import literal_eval
from gensim import corpora
import os
#from data_ingestion import read_txt_content, preprocess_data, get_date, get_ref, get_files_list
from kpi import load_corpus_model, preprocess_data, load_lda_model, calculate_coherence


In [2]:
folder_name = 'CI_newspaper_subcorpora'
pub_refs = ["2012271201", "sn85054967", "sn93053873", "sn85066408", "sn85055164", "sn84037024", "sn84037025", "sn84020351", "sn86092310", "sn92051386"]
pub_names = ["Cronaca_Sovversiva", "Il_Patriota", "L'Indipendente", "L'Italia", "La_Libera_Parola", "La_Ragione", "La_Rassegna", "La_Sentinella", "La_Sentinella_del_West", "La_Tribuna_del_Connecticut"]
date_ref_1 = "1903-06-06"
date_ref_2 = "1919-05-01"

In [3]:
corpus_model_file = 'corpus_model.csv'
current_directory = os.getcwd()
lda_model_file = os.path.join(current_directory, 'lda_model')  # Construct the path to the LDA model file
bigrams = load_corpus_model(corpus_model_file)
id2word, corpus = preprocess_data(bigrams)
lda = load_lda_model(lda_model_file)

In [4]:
topic_print_model = lda.print_topics(num_words=30)

In [6]:
df_topic_print_model = pd.DataFrame(topic_print_model, columns=['topic_id','words'])

In [11]:
topic_print_model

[(0,
  '0.001*"pace" + 0.001*"america" + 0.001*"francisco" + 0.001*"pubblico" + 0.001*"loggia" + 0.001*"sera" + 0.001*"vero" + 0.001*"patria" + 0.001*"morte" + 0.001*"capo" + 0.001*"alia" + 0.001*"figli" + 0.001*"società" + 0.001*"venne" + 0.001*"legge" + 0.001*"dollari" + 0.001*"colonia" + 0.001*"vittoria" + 0.001*"comitato" + 0.001*"londra" + 0.001*"cuore" + 0.001*"storia" + 0.001*"disse" + 0.001*"diritto" + 0.001*"fatti" + 0.001*"direttore" + 0.001*"altra" + 0.001*"delia" + 0.001*"milano" + 0.001*"lire"'),
 (1,
  '0.002*"pace" + 0.002*"austriaci" + 0.002*"tedeschi" + 0.001*"wilson" + 0.001*"alleati" + 0.001*"nemico" + 0.001*"congresso" + 0.001*"vittoria" + 0.001*"nazioni" + 0.001*"questione" + 0.001*"america" + 0.001*"londra" + 0.001*"politica" + 0.001*"americani" + 0.001*"ltalia" + 0.001*"diritto" + 0.001*"americano" + 0.001*"legge" + 0.001*"marzo" + 0.001*"ufficiali" + 0.001*"patria" + 0.001*"francese" + 0.001*"italiane" + 0.001*"presidente_wilson" + 0.001*"lotta" + 0.001*"militar

In [20]:
print(topic_print_model[0][0], topic_print_model[0][1])


0 0.001*"pace" + 0.001*"america" + 0.001*"francisco" + 0.001*"pubblico" + 0.001*"loggia" + 0.001*"sera" + 0.001*"vero" + 0.001*"patria" + 0.001*"morte" + 0.001*"capo" + 0.001*"alia" + 0.001*"figli" + 0.001*"società" + 0.001*"venne" + 0.001*"legge" + 0.001*"dollari" + 0.001*"colonia" + 0.001*"vittoria" + 0.001*"comitato" + 0.001*"londra" + 0.001*"cuore" + 0.001*"storia" + 0.001*"disse" + 0.001*"diritto" + 0.001*"fatti" + 0.001*"direttore" + 0.001*"altra" + 0.001*"delia" + 0.001*"milano" + 0.001*"lire"


In [25]:
dic = { topic_print_model[i][0] : topic_print_model[i][1]  for i in range (7)}

In [26]:
dic

{0: '0.001*"pace" + 0.001*"america" + 0.001*"francisco" + 0.001*"pubblico" + 0.001*"loggia" + 0.001*"sera" + 0.001*"vero" + 0.001*"patria" + 0.001*"morte" + 0.001*"capo" + 0.001*"alia" + 0.001*"figli" + 0.001*"società" + 0.001*"venne" + 0.001*"legge" + 0.001*"dollari" + 0.001*"colonia" + 0.001*"vittoria" + 0.001*"comitato" + 0.001*"londra" + 0.001*"cuore" + 0.001*"storia" + 0.001*"disse" + 0.001*"diritto" + 0.001*"fatti" + 0.001*"direttore" + 0.001*"altra" + 0.001*"delia" + 0.001*"milano" + 0.001*"lire"',
 1: '0.002*"pace" + 0.002*"austriaci" + 0.002*"tedeschi" + 0.001*"wilson" + 0.001*"alleati" + 0.001*"nemico" + 0.001*"congresso" + 0.001*"vittoria" + 0.001*"nazioni" + 0.001*"questione" + 0.001*"america" + 0.001*"londra" + 0.001*"politica" + 0.001*"americani" + 0.001*"ltalia" + 0.001*"diritto" + 0.001*"americano" + 0.001*"legge" + 0.001*"marzo" + 0.001*"ufficiali" + 0.001*"patria" + 0.001*"francese" + 0.001*"italiane" + 0.001*"presidente_wilson" + 0.001*"lotta" + 0.001*"militare" + 0.